# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем библиотеки в следующей ячейке.

In [29]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer 
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
import numpy as np
import warnings
warnings.simplefilter("ignore")

Просмотрим файл:

In [21]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


Нужно лемматизировать, очистить от знаков комментарии, сделаем это в следующей ячейке:

In [22]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text).split()
    return " ".join(text)

df['lemm_text'] = df['text'].apply(lemmatize_text)
df['lemm_text'] = df['text'].apply(clear_text)

Далее почистим от стоп слов.

In [23]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Разделим датасет на выборки, а так же вычислим TF-IDF для корпуса текстов.

In [24]:
target = df['toxic']
features = df['lemm_text']
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

In [25]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train = count_tf_idf.fit_transform(features_train) 
features_valid = count_tf_idf.transform(features_valid)

**Вывод**
 
Данные обработаны и готовы к обучению моделей.

## Обучение

### Логистическая регрессия

In [26]:
model_log_r = LogisticRegression(random_state=12345, solver='liblinear', penalty = 'l1')
model_log_r.fit(features_train, target_train) 
predictions_valid_log_r = model_log_r.predict(features_valid)
probabilities_valid = model_log_r.predict_proba(features_valid)
print('f1:',f1_score(target_valid, predictions_valid_log_r))

f1: 0.7746947835738069


### Решающее дерево

In [7]:
parametrs = {'max_depth': range (1,70)}
model_dtc = DecisionTreeClassifier(random_state=12345)
grid_dtc = GridSearchCV(model_dtc, parametrs, cv=5, scoring='f1')
grid_dtc.fit(features_train, target_train)
grid_dtc.best_params_

{'max_depth': 69}

In [27]:
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=69)
model_dtc.fit(features_train, target_train)
predictions_valid_dtc = model_dtc.predict(features_valid)
probabilities_valid = model_dtc.predict_proba(features_valid)
print('f1:',f1_score(target_valid, predictions_valid_dtc))

f1: 0.7112842632530958


## Выводы

Лучше всего себя показала модель логистическая регрессия, её результат по метрике F1 = 0,774 против 0,711 у решающего дерева. Задачей проекта было добиться результата выше 0,75, модель так же преодолела этот порог, соответственно, для нашей задачи отбора токсичных комментариев лучше всего подойдет именно она.